In [106]:
import pandas as pd
import regex as re
from textblob import TextBlob
from nltk.corpus import stopwords #stopwords
from nltk import pos_tag,pos_tag_sents #POS tags
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
#from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV 
#GridSearchCV-advanced form of cross-validation
import pickle

In [148]:
data= pd.read_csv("spam.csv")
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


In [150]:
data.describe(include='all')

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2


In [151]:
data["Unnamed: 2"].isna().sum()

5522

In [152]:

data['Unnamed: 2'].fillna('',inplace=True)
data['a']=data['Unnamed: 2']
data['Unnamed: 3'].fillna('',inplace=True)
data['b']=data['Unnamed: 3']
data['Unnamed: 4'].fillna('',inplace=True)
data['c']=data['Unnamed: 4']
#data['v2'] =data[data.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
data['v2'] = data['v2']+data['a']+data['b']+data['c']
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

In [153]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1    5572 non-null object
v2    5572 non-null object
a     5572 non-null object
b     5572 non-null object
c     5572 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


,v1,v2,a,b,c
0,ham,"Go until jurong point, crazy.. Available only ...",,,
1,ham,Ok lar... Joking wif u oni...,,,
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,,,
3,ham,U dun say so early hor... U c already then say...,,,
4,ham,"Nah I don't think he goes to usf, he lives aro...",,,
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,,,
5568,ham,Will Ì_ b going to esplanade fr home?,,,
5569,ham,"Pity, * was in mood for that. So...any other s...",,,
5570,ham,The guy did some bitching but I acted like i'd...,,,


In [154]:
data['org_text']=data['v2']
data['org_text']

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: org_text, Length: 5572, dtype: object

In [155]:
#Convert to lower case
data['v2']=data['v2'].str.lower()
#Remove URL's
data['v2']=data['v2'].str.replace('http\S+|www.\S+','',case=False)
#Remove numbers
data['v2']=data['v2'].str.replace('\d+','')
#Remove whitespaces
data['v2']=data['v2'].str.replace('\W+',' ')
data['v2']=data['v2'].str.replace('\s{2,}',' ')
data['v2']=data['v2'].str.replace('\s{3,}',' ')

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [156]:
from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    #dictionary.get(keyname, value)
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [157]:
data['v2']=data.v2.apply(lemmatize_with_postag)

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [158]:
stop_words_nltk=nltk.corpus.stopwords.words('english')
for word in stop_words_nltk :
    data['v2'] = data['v2'].replace(to_replace=r'\b%s\b'%word, value="",regex=True)

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [159]:
data['v2']=data['v2'].map(lambda x:x.strip())
data['v2']

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       go  jurong point crazy available   bugis n gre...
1                                 ok lar joking wif u oni
2       free entry   wkly comp  win fa cup final tkts ...
3                   u dun say  early hor u c already  say
4              nah    think  go  usf  live around  though
                              ...                        
5567    nd time   try contact u u  win  å pound prize ...
5568                           ì_ b go  esplanade fr home
5569                          pity   mood      suggestion
5570    guy   bitching   act like    interested  buy s...
5571                                    rofl  true   name
Name: v2, Length: 5572, dtype: object

In [160]:
#data['freq'] = data.groupby('v2')['v2'].transform('count')
#data
from sklearn.feature_extraction.text import CountVectorizer
data['v1'] = data.v1.map({'ham':0, 'spam':1})
X = data.v2
y = data.v1
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=100,test_size=0.2)
vect = CountVectorizer(ngram_range=(2,2))

C:\Users\ARAVIND NACHIAPPAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [161]:
X.isna().sum()

0

In [162]:

X_train_vector= vect.fit_transform(X_train)
X_test_vector = vect.transform(X_test)

In [163]:
model_multinom=MultinomialNB()
model_multinom.fit(X_train_vector,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [164]:

from sklearn.externals import joblib
joblib.dump(model_multinom,'multinom_obj.sav')

['multinom_obj.sav']

In [165]:
joblib.dump(vect,'vect_obj.sav')

['vect_obj.sav']

In [166]:
y_pred = model_multinom.predict(X_test_vector)

In [167]:
from sklearn import metrics
metrics.accuracy_score(Y_test, y_pred)

0.9820627802690582

In [168]:
metrics.confusion_matrix(Y_test, y_pred)

array([[966,   4],
       [ 16, 129]], dtype=int64)